In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('data.csv', parse_dates=[1]).sort_values(by = "date")
df['date'] = df['date'].dt.to_period('M')
df = df[277:]

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180537 entries, 113114 to 180813
Data columns (total 37 columns):
 #   Column                           Non-Null Count   Dtype    
---  ------                           --------------   -----    
 0   optid                            180537 non-null  int64    
 1   date                             180537 non-null  period[M]
 2   secid                            180537 non-null  int64    
 3   cp_flag                          180537 non-null  object   
 4   strike                           180537 non-null  float64  
 5   bid                              180537 non-null  float64  
 6   ask                              180537 non-null  float64  
 7   volume                           180537 non-null  float64  
 8   openint                          180537 non-null  float64  
 9   impvol                           180537 non-null  float64  
 10  delta                            180537 non-null  float64  
 11  gamma                            1

In [4]:
def get_summary(df_col):
    print(f'\nSummary of {df_col.name}:')
    print(f'Mean: {np.round( df_col.mean(),2)}')
    print(f'Median: {np.round( df_col.median(),2)}')
    print(f'Standard deviation: {np.round( df_col.std(),2)}')
    print(f'Number of observations: {np.round( df_col.count(),2)}')    

In [5]:
def add_months(date ,period_to_add=1):
    return (datetime.strptime(date, '%Y-%m') + relativedelta(months=period_to_add)).strftime('%Y-%m')

In [6]:
features = [ 'midprice', 'mness', 'ttm', 'embed_lev', 'impvol', 'gamma', 'vega', 'theta']

In [7]:
print('\nSummary statistics for Call options\n')
df[df["cp_flag"] == 'C'][features].apply(lambda x: get_summary(x), axis=0);


Summary statistics for Call options


Summary of midprice:
Mean: 91.46
Median: 37.39
Standard deviation: 163.66
Number of observations: 69044

Summary of mness:
Mean: 0.49
Median: 0.31
Standard deviation: 1.3
Number of observations: 69044

Summary of ttm:
Mean: 208.87
Median: 91.0
Standard deviation: 224.3
Number of observations: 69044

Summary of embed_lev:
Mean: 22.17
Median: 14.24
Standard deviation: 21.85
Number of observations: 69044

Summary of impvol:
Mean: 0.21
Median: 0.18
Standard deviation: 0.15
Number of observations: 69044

Summary of gamma:
Mean: 0.0
Median: 0.0
Standard deviation: 0.0
Number of observations: 69044

Summary of vega:
Mean: 240.76
Median: 176.03
Standard deviation: 230.11
Number of observations: 69044

Summary of theta:
Mean: -56.13
Median: -43.84
Standard deviation: 51.24
Number of observations: 69044


In [8]:
print('\nSummary statistics for Put options\n')

df[df["cp_flag"] == 'P'][features].apply(lambda x: get_summary(x), axis=0);


Summary statistics for Put options


Summary of midprice:
Mean: 71.67
Median: 14.25
Standard deviation: 154.16
Number of observations: 111493

Summary of mness:
Mean: -0.9
Median: -1.03
Standard deviation: 1.43
Number of observations: 111493

Summary of ttm:
Mean: 202.65
Median: 91.0
Standard deviation: 222.21
Number of observations: 111493

Summary of embed_lev:
Mean: 14.65
Median: 11.78
Standard deviation: 10.6
Number of observations: 111493

Summary of impvol:
Mean: 0.3
Median: 0.26
Standard deviation: 0.18
Number of observations: 111493

Summary of gamma:
Mean: 0.0
Median: 0.0
Standard deviation: 0.0
Number of observations: 111493

Summary of vega:
Mean: 178.92
Median: 105.07
Standard deviation: 205.27
Number of observations: 111493

Summary of theta:
Mean: -49.22
Median: -36.21
Standard deviation: 49.53
Number of observations: 111493


In [9]:
features += ['strike', 'spot_close', 'divrate', 'vix', 'short_rate', 'delta', 'date']# + ['optid']
train_features = features[1:-1]

In [10]:
clear_df = df[features]

In [11]:
start_date = '2007-01'

y_valid = clear_df[clear_df['date'] > start_date]['midprice']

In [12]:
ols_predictions = np.array([])
ols_r_2 = np.array([])

In [13]:
for current_date in df[df['date'] > start_date]['date'].unique():
    
    x_train = clear_df[clear_df['date'] < current_date][train_features]
    y_train = clear_df[clear_df['date'] < current_date]['midprice']

    x_test = clear_df[clear_df['date'] == current_date][train_features]
    y_test = clear_df[clear_df['date'] == current_date]['midprice']
    
    ols = LinearRegression()
    ols.fit(x_train, y_train)
    
    y_pred = ols.predict(x_test)
    
    # Write predictions to array
    ols_predictions = np.append(ols_predictions, y_pred)
    
    # Write R^2 to array
    ols_r_2 = np.append(ols_r_2, ols.score(x_train, y_train))

In [19]:
np.mean(ols_r_2)

129

In [17]:
# MSE
np.sqrt(mean_squared_error(y_test, y_pred))

186.35256406047677